--- Day 19: Aplenty ---

The Elves of Gear Island are thankful for your help and send you on your way. They even have a hang glider that someone stole from Desert Island; since you're already going that direction, it would help them a lot if you would use it to get down there and return it to them.

As you reach the bottom of the relentless avalanche of machine parts, you discover that they're already forming a formidable heap. Don't worry, though - a group of Elves is already here organizing the parts, and they have a system.

To start, each part is rated in each of four categories:

- x: Extremely cool looking
- m: Musical (it makes a noise when you hit it)
- a: Aerodynamic
- s: Shiny

Then, each part is sent through a series of workflows that will ultimately accept or reject the part. Each workflow has a name and contains a list of rules; each rule specifies a condition and where to send the part if the condition is true. The first rule that matches the part being considered is applied immediately, and the part moves on to the destination described by the rule. (The last rule in each workflow has no condition and always applies if reached.)

Consider the workflow ex{x>10:one,m<20:two,a>30:R,A}. This workflow is named ex and contains four rules. If workflow ex were considering a specific part, it would perform the following steps in order:

- Rule "x>10:one": If the part's x is more than 10, send the part to the workflow named one.
- Rule "m<20:two": Otherwise, if the part's m is less than 20, send the part to the workflow named two.
- Rule "a>30:R": Otherwise, if the part's a is more than 30, the part is immediately rejected (R).
- Rule "A": Otherwise, because no other rules matched the part, the part is immediately accepted (A).

If a part is sent to another workflow, it immediately switches to the start of that workflow instead and never returns. If a part is accepted (sent to A) or rejected (sent to R), the part immediately stops any further processing.

The system works, but it's not keeping up with the torrent of weird metal shapes. The Elves ask if you can help sort a few parts and give you the list of workflows and some part ratings (your puzzle input). For example:

```
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}
```

```
{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
```
The workflows are listed first, followed by a blank line, then the ratings of the parts the Elves would like you to sort. All parts begin in the workflow named in. In this example, the five listed parts go through the following workflows:

```
{x=787,m=2655,a=1222,s=2876}: in -> qqz -> qs -> lnx -> A
{x=1679,m=44,a=2067,s=496}: in -> px -> rfg -> gd -> R
{x=2036,m=264,a=79,s=2244}: in -> qqz -> hdj -> pv -> A
{x=2461,m=1339,a=466,s=291}: in -> px -> qkq -> crn -> R
{x=2127,m=1623,a=2188,s=1013}: in -> px -> rfg -> A
```

Ultimately, three parts are accepted. Adding up the x, m, a, and s rating for each of the accepted parts gives 7540 for the part with x=787, 4623 for the part with x=2036, and 6951 for the part with x=2127. Adding all of the ratings for all of the accepted parts gives the sum total of 19114.

Sort through all of the parts you've been given; what do you get if you add together all of the rating numbers for all of the parts that ultimately get accepted?

Your puzzle answer was 434147.

In [ ]:
import re

def read_input(filePath):
    with open(filePath, 'r') as file:
        return file.read()
    
def read_workflows(text):
    workflow_pattern = r'(\w+)\{(.*),(\w+)\}';
    rule_pattern = r'(\w)([<>])(\d+):(\w+)';
    lines = re.findall(workflow_pattern, text)
    return {name: [*[(lhs, cond, int(rhs), outcome) for (lhs, cond, rhs, outcome) in re.findall(rule_pattern, rules)], fallback] for (name, rules, fallback) in lines}

def read_ratings(text):
    ratings_pattern = r'\{x=(\d+),m=(\d+),a=(\d+),s=(\d+)\}'
    matches = re.findall(ratings_pattern, text)
    return [{'x': int(x), 'm': int(m), 'a': int(a), 's': int(s)} for (x, m, a, s) in matches]

def eval_rating(workflows, workflow_key, rating):
    #
    # Example workflows
    #
    #   in{s<7:px,qqz}
    #   px{a<6:A,m<9:R,rfg}
    #
    # Corresponding tree
    #
    #              in
    #             /  \
    #          s<7    \    
    #           /      \
    #          px      qqz
    #         / | \   / | \
    #      a<6 m<9 \
    #       /   |   \
    #     (A)   (R)  rfg
    #
    # First check if we are at a leaf node
    #
    if workflow_key == 'A' or workflow_key == 'R':
        return workflow_key
    #
    # Loop over child rules to get to new nodes
    #
    else:
        workflow = workflows[workflow_key]
        for rule in workflow:
            if isinstance(rule, tuple):
                (lhs, cond, rhs, outcome) = rule
                if cond == '<' and rating[lhs] < rhs:
                    return eval_rating(workflows, outcome, rating)
                elif cond == '>' and rating[lhs] > rhs:
                    return eval_rating(workflows, outcome, rating)
            else:
                return eval_rating(workflows, rule, rating)

input = read_input('sample.txt')
workflows = read_workflows(input)
ratings = read_ratings(input)

answer = 0
for rating in ratings:
    outcome = eval_rating(workflows, 'in', rating)
    if outcome == 'A':
        answer += sum(rating.values())

print('----------------')
print(answer)
print('----------------')

--- Part Two ---

Even with your help, the sorting process still isn't fast enough.

One of the Elves comes up with a new plan: rather than sort parts individually through all of these workflows, maybe you can figure out in advance which combinations of ratings will be accepted or rejected.

Each of the four ratings (x, m, a, s) can have an integer value ranging from a minimum of 1 to a maximum of 4000. Of all possible distinct combinations of ratings, your job is to figure out which ones will be accepted.

In the above example, there are 167409079868000 distinct combinations of ratings that will be accepted.

Consider only your list of workflows; the list of part ratings that the Elves wanted you to sort is no longer relevant. How many distinct combinations of ratings will be accepted by the Elves' workflows?

Your puzzle answer was 136146366355609.

In [ ]:
import copy
from math import prod


def generate_ranges(workflows, workflow_key, workflow_ranges):
    #
    # Example workflows
    #
    #   in{s<7:px,qqz}
    #   px{a<6:A,m<9:R,rfg}
    #
    # Corresponding tree (Assuming max value of 10)
    #
    #              in
    #             /  \
    #          s<7    \    
    #           /      \
    #          px      qqz
    #         / | \   / | \
    #      a<6 m<9 \
    #       /   |   \
    #     (A)   (R)  rfg
    #
    # Assume the initial ranges
    #
    #   a < [1, 10], m < [1, 10], s < [1, 10], x < [1, 10].
    #
    # As we move down the tree we can update the ranges for each node wich
    # will then give us the possible combinations for that node. For instance
    # the node at (A) has:
    #
    #   a < [1, 5], m < [1, 10], s < [1, 6], x < [1, 10].
    #
    # By combinginh these we get 5 * 10 * 6 * 10 = 3000 possible combinations
    # for this node. Summing all the accepted nodes, give the final answer.
    #
    if workflow_key == 'A' or workflow_key == 'R':
        yield (workflow_key, workflow_ranges)
    else:
        workflow = workflows[workflow_key]
        remaining_ranges = copy.deepcopy(workflow_ranges)
        for rule in workflow:
            if isinstance(rule, tuple):
                (lhs, cond, rhs, outcome) = rule
                if cond == '<':
                    rule_ranges = copy.deepcopy(remaining_ranges)
                    rule_ranges[lhs] = (remaining_ranges[lhs][0], min(rhs - 1, remaining_ranges[lhs][1]))
                    remaining_ranges[lhs] = (max(rhs, remaining_ranges[lhs][0]), remaining_ranges[lhs][1])
                    yield from generate_ranges(workflows, outcome, rule_ranges)
                elif cond == '>':
                    rule_ranges = copy.deepcopy(remaining_ranges)
                    rule_ranges[lhs] = (max(rhs + 1, remaining_ranges[lhs][0]), remaining_ranges[lhs][1])
                    remaining_ranges[lhs] = (remaining_ranges[lhs][0], min(rhs, remaining_ranges[lhs][1]))
                    yield from generate_ranges(workflows, outcome, rule_ranges)
            else:
                yield from generate_ranges(workflows, rule, remaining_ranges)

input = read_input('input.txt')
workflows = read_workflows(input)

answer = 0
inklusive_ranges = {'a': (1, 4000), 'm': (1, 4000), 's': (1, 4000), 'x': (1, 4000)}
for (outcome, ranges) in generate_ranges(workflows, 'in', inklusive_ranges):
    print(outcome, ranges)
    if outcome == 'A':
        combinations = prod([max - min + 1 for (min, max) in ranges.values()])
        answer += combinations

print('----------------')
print(answer)
print('----------------')